In [191]:
import pickle
import re

with open('chgk/players.pkl', 'rb') as f:
    players = pickle.load(f)
with open('chgk/results.pkl', 'rb') as f:
    results = pickle.load(f)
with open('chgk/tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

### 1. Сформируем трейн и тест выборки

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl)

- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.

In [192]:
tour_train = {}
tour_test = {}
team_test = {}
team_train = {}
result_preproc = {}
for _, tour in tournaments.items():
    if tour['dateStart'][:4] in ('2020', '2019'):
        tour_id = int(tour['id'])
        filtred_teams = []
        qs_count = tour['questionQty']
        qs_count = sum([int(q) for q in qs_count.values()])
        filtred_tour_teams = [team for team in results[tour_id] if 'mask' in team.keys() \
                         and team['mask'] and len(team['teamMembers']) > 0 \
                         and len(team['mask']) == qs_count \
                         and len(re.findall(r'[0-9]', team['mask'])) == qs_count]
        if len(filtred_tour_teams) > 0:
            if tour['dateStart'][:4] == '2019':
                tour_train[tour_id] = (tour['name'], filtred_tour_teams)
#                 team_train[tour_id] = filtred_teams
            else:
                tour_test[tour_id] = (tour['name'], filtred_tour_teams)
#                 team_test[tour_id] = filtred_teams

            result_preproc[tour_id] = results[tour_id]

Размер получившегося датасета туров. Дальше к каждой выборке туров мы подтянем данные о составе команды, игроках, вопросах и успешных ответах

In [193]:
len(tour_train), len(tour_test)

(614, 159)

In [306]:
train_players = {}
for tour_id, (tour_name, filtred_tour_teams) in tour_train.items():
    for team in filtred_tour_teams:
        position = team['position']
        player_ids = [mem['player']['id'] for mem in team['teamMembers']]
        questions = [int(q) for q in team['mask']]
        answers = [int(q) for q in team['mask']]
        for pid in player_ids:
            if pid not in train_players:
                train_players[pid] = {}
            train_players[pid][tour_id] = (team['team']['id'], team['position'], answers)
            
plrs, trs, teams_list, ans, pos, qs = [], [], [], [], [], []
for player, tour_id in train_players.items():
    for tour,  (teams_id, position, answers) in tour_id.items():
        for i, answer in enumerate(answers):
            plrs.append(player)
            trs.append(tour)
            ans.append(answer)
            teams_list.append(teams_id)
            pos.append(position)
            qs.append(str(tour) + '_' + str(i))
dict_test = {'tour_id':trs, 'team_id':teams_list, 'position':pos, 'player_id':plrs, 'question_id':qs,  'answer':ans}

In [307]:
# Нас интересуют только игроки, которые есть в обучающей выборке
test_players = {}
for tour_id, (tour_name, filtred_tour_teams) in tour_test.items():
    for team in filtred_tour_teams:
        player_ids = [mem['player']['id'] for mem in team['teamMembers']]
        answers = [int(q) for q in team['mask']]
        for pid in player_ids:
            if pid not in train_players:
                continue
            if pid not in test_players:
                test_players[pid] = {}
            test_players[pid][tour_id] = (team['team']['id'], team['position'], answers)

plrs, trs, teams_list, ans, pos, qs = [], [], [], [], [], []
for player, tour_id in test_players.items():
    for tour, (teams_id, position, answers) in tour_id.items():
        for i, answer in enumerate(answers):
            plrs.append(player)
            trs.append(tour)
            ans.append(answer)
            teams_list.append(teams_id)
            pos.append(position)
            qs.append(str(tour) + '_' + str(i))
dict_test = {'tour_id':trs, 'team_id':teams_list, 'position':pos, 'player_id':plrs, 'question_id':qs,  'answer':ans}

In [308]:
# Train Dataset
df = pd.DataFrame(dict_train)

# Test Dataset
df_test = pd.DataFrame(dict_test)

In [310]:
df_test.head(10)

,tour_id,team_id,position,player_id,question_id,answer
0,5414,66120,1.0,18490,5414_0,1
1,5414,66120,1.0,18490,5414_1,1
2,5414,66120,1.0,18490,5414_2,1
3,5414,66120,1.0,18490,5414_3,1
4,5414,66120,1.0,18490,5414_4,1
5,5414,66120,1.0,18490,5414_5,1
6,5414,66120,1.0,18490,5414_6,1
7,5414,66120,1.0,18490,5414_7,1
8,5414,66120,1.0,18490,5414_8,1
9,5414,66120,1.0,18490,5414_9,1


Датасеты по количеству уникальных игроков:

In [311]:
df.player_id.nunique(), df_test.player_id.nunique()

(55624, 22169)

### 2. Построение Baseline модели
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков
- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Будем использовать статистику по игроку и каждому вопросу в туре в качестве признаков, таргет переменной будет значение (0, 1) - ответил ли данный человек (в составе какой-либо команды) на вопрос или нет. Закодируем все признаки игроков и вопросов one-hot кодированием

In [126]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
X_train = encoder.fit_transform(df[['player_id', 'question_id']])
X_test = encoder.transform(df_test[['player_id', 'question_id']])
y_train = df['answer']
y_test = df_test['answer']

In [128]:
X_train.shape, X_test.shape

((14678548, 85781), (3427173, 85781))

In [130]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='saga')
lr.fit(X_train, y_train)

LogisticRegression(solver='saga')

In [131]:
# saving the model
pickle.dump(lr, open('lr_model.pkl', 'wb'))

In [172]:
players_weights = {id: w for (id, w) in zip(encoder.categories_[0], lr.coef_[0][:len(train_players.keys())])}
questions_weights = {id: w for (id, w) in zip(encoder.categories_[1], lr.coef_[0][len(train_players.keys()):])}
ranking_df = pd.DataFrame(train_players.keys(), columns=['player_id'])
names = [players[player_id]['name'] + ' ' + players[player_id]['surname'] + ' ' + players[player_id]['patronymic'] for player_id in train_players.keys()]
ranking_df['name'] = names
ranking_df['weight'] = ranking_df['player_id'].map(players_weights)
ranking_df['rating'] = ranking_df['weight'].rank(ascending=False)
ranking_df.sort_values(by='rating', inplace=True, ignore_index=True)
ranking_df.head(20)

,player_id,name,weight,rating
0,27403,Максим Руссо Михайлович,4.071555,1.0
1,4270,Александра Брутер Владимировна,3.935011,2.0
2,28751,Иван Семушин Николаевич,3.887821,3.0
3,27822,Михаил Савченков Владимирович,3.855391,4.0
4,30270,Сергей Спешков Леонидович,3.761352,5.0
5,30152,Артём Сорожкин Сергеевич,3.755803,6.0
6,18036,Михаил Левандовский Ильич,3.620864,7.0
7,20691,Станислав Мереминский Григорьевич,3.616413,8.0
8,87637,Антон Саксонов Владимирович,3.546105,9.0
9,22799,Сергей Николенко Игоревич,3.541687,10.0


### 3. Предсказания результатов нового турнира с известными составами, но неизвестными вопросами
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Для успеха команды достаточно, чтобы один из игроков назвал правильный ответ. Поэтому вероятность команды правильно ответить на вопрос = вероятности, что хотя бы один из игроков правильно ответит на вопрос, воспользуемся данной вероятностью при ранжировании команд

In [356]:
X_test = encoder.transform(df_test[['player_id', 'question_id']])
y_pred = lr.predict_proba(X_test)[:, 1]
df_score = df_test[['tour_id', 'team_id']].copy()
df_score['1-p'] = 1 - y_pred
df_score = df_score.groupby(['tour_id',  'team_id']).prod().reset_index()
df_score['rating'] = df_score.groupby('tour_id')['1-p'].rank('dense')

team_rating = pd.merge(df_score[['tour_id', 'team_id', 'rating']].drop_duplicates(), df_test[['tour_id', 'team_id', 'position']].drop_duplicates(), on=['tour_id', 'team_id'])

In [359]:
# посчитаем ранговые корреляции между ранжированием в результатах турнира и предсказанных моделью
from scipy import stats

spearman_corr = []
kendall_corr = []
for tour in list(team_rating['tour_id'].values):
    df = team_rating[team_rating['tour_id'] == tour]
    s_corr = stats.spearmanr(df['rating'], df['position']).correlation
    k_corr = stats.kendalltau(df['rating'], df['position']).correlation
    spearman_corr.append(s_corr)
    kendall_corr.append(k_corr)

In [358]:
sp_corr = np.mean(spearman_corr)
kd_corr = np.mean(kendall_corr)
print('Корреляция Спирмена:', round(sp_corr, 3))
print('Корреляция Кендалла:', round(kd_corr, 3))

Корреляция Спирмена: 0.829
Корреляция Кендалла: 0.658
